<a href="https://colab.research.google.com/github/EltonWolf/GoogleEarthPythonBootcampAPI-/blob/main/Composting_Mexico_Landsat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=bq0hajtSJIzKzPrk9f05hnneUVwLNC1xu-lylNUzVzM&tc=OBl5980XDEPRw_SZVtATP3zGWwHIXNUNZdMfu6ACziA&cc=MEvHjzAj1pIal_a0tuy2wVOqHKbyHWkJPdY1efkCYrU

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXlNPWBneYmYBWD7WHC1Z0Ev4aGoOpghS7X3mtBLYkzbGn-k_1y6Bg4

Successfully saved authorization token.


In [2]:
countries = ee.FeatureCollection("FAO/GAUL/2015/level0")
Mexico = countries.filter(ee.Filter.eq("ADM0_NAME", "Mexico"))

In [3]:
landsat = (
    ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
    .filterDate('2021-01-01', '2022-02-01')
    .filterBounds(Mexico)
)

In [4]:
# Select bands needed for composite
optical_bands = ['SR_B4', 'SR_B3', 'SR_B2']  # Adjust band names as needed

# Create a function to apply scaling factors
def apply_scale_factors(image):
    optical = image.select(optical_bands).multiply(0.0000275).add(-0.2)
    return image.addBands(optical, None, True)

landsat_scaled = landsat.map(apply_scale_factors)

In [6]:
# Create a composite
composite = ee.Image(landsat_scaled.median())

#Visualisation
rgbVis = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],  # Adjust band names as needed
    'min': 0.0,
    'max': 0.3,
}

In [9]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object.
my_map = folium.Map(location=[19.4326, -99.1332], zoom_start=6)

In [10]:
# Add the layer to the map object.
my_map.add_ee_layer(composite, rgbVis, "RGB")

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)